## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [4]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            file_read = path.join(root, name)
            dir_name = (os.path.splitext(os.path.basename(root))[0])
            print(file_read)
            
            filename = (os.path.splitext(os.path.basename(file_read))[0])
            print(filename)
            with open(file_read,'r') as inf:
                query = json.load(inf)
                
                kcresult = submit_to_ars(query)
                
                sleep(400)
                
                result_status = retrieve_ars_results(kcresult)
                
        
                dict_workflows[filename] = kcresult
                            
                sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=b93fa28a-9446-4283-a86e-fc07d28a2861
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 140
Done
ara-bte Done 18
Done
ara-unsecret Done 99
Done
kp-genetics Done 0
Done
kp-molecular Done 97
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Done
ara-robokop Done 30
Done
kp-chp Done 20
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=ab214d37-ac26-45c4-bfc0-6d4a14e09490
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 433
Done
ara-bte Done 78
Done
ara-unsecret Done 0
Done
kp-genetics Done 0
Done
kp-molecular Done 164
Done
ara-explanatory Done 0
Done
ara-improving Done 26
Error
kp-cam E

https://arax.ncats.io/?source=ARS&id=f34d673d-7eda-410c-a9c4-15ed652825d1
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 0
Done
ara-bte Done 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Done
ara-robokop Done 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-icees-dili ARS Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi.json
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
https://arax.ncats.io/?source=ARS&id=6ea4e662-e818-4f04-87fb-d2889b24e160
Done
Done
ara-aragorn Done 0
Done
ara-arax Done 500
Done
ara-bte Done 0
Done
ara-unsecret Done 0
Unknown
kp-genetics Unknown 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving ARS Error 0
Error


<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [5]:
dict_workflows

{'A.3_KCNMA1': 'b93fa28a-9446-4283-a86e-fc07d28a2861',
 'A.8_EGFR_simple': 'ab214d37-ac26-45c4-bfc0-6d4a14e09490',
 'A.0_RHOBTB2_direct': 'ac9d3422-5c5f-49a5-879c-f166a76597ef',
 'A.2_RHOBTB2_twohop': '2096d6b9-26c6-490b-a5b4-9e9123284785',
 'A.9_EGFR_advanced': 'e8b61760-8126-4290-951f-66e76545135f',
 'A.1_RHOBTB2': '2093c1a9-5e3f-4bca-8303-1421626ac426',
 'B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi': '6db214a6-8af0-410c-b5ee-fc8fb6fb5da0',
 'B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': 'dcdadbd4-b298-436c-9e0e-35357213e06d',
 'B.4_one-hop-gene-biological-process-or-activity_trapi': '3d105170-e33a-4e7c-8645-39cdf1bb4c0e',
 'B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi': '8830b3e9-928e-4890-8bd4-a085fdae129f',
 'B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi': '54f84c1b-0785-4473-8d37-f5638ea0aa18',
 'B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi': 'f008ad98-4fae-480a-9155-bec2b5762c94',
 'B.3_DILI-one

In [6]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1 b93fa28a-9446-4283-a86e-fc07d28a2861
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 140
Done
ara-bte Done 18
Done
ara-unsecret Done 99
Done
kp-genetics Done 0
Done
kp-molecular Done 97
Done
ara-explanatory Done 0
Done
ara-improving Done 5
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Done
ara-robokop Done 30
Done
kp-chp Done 20
Done
kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
A.8_EGFR_simple ab214d37-ac26-45c4-bfc0-6d4a14e09490
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 433
Done
ara-bte Done 78
Done
ara-unsecret Done 0
Done
kp-genetics Done 0
Done
kp-molecular Done 164
Done
ara-explanatory Done 0
Done
ara-improving Done 26
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Done
ara-robokop Done 83
Done
kp-chp Done 0
Error
kp-cohd Error 0
Done
kp-icees-dili Done 0
A.0_RHOBTB2_direct ac9d3422-5c5f-49a5-879c-f16

Done
Error
ara-aragorn Error 0
Done
ara-arax Done 500
Error
ara-bte Error 0
Error
ara-unsecret Error 0
Error
kp-genetics Error 0
Error
kp-molecular Error 0
Done
ara-explanatory Done 0
Done
ara-improving Done 22
Error
kp-cam Error 0
Error
kp-textmining Error 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Done
ara-robokop Done 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Error
kp-icees-dili Error 0
D.3_ssri-heart-disease-one-hop e2b484a8-2c10-4ccd-8068-ff0eb911d768
Done
Error
ara-aragorn Error 0
Done
ara-arax Done 10
Done
ara-bte Done 12
Error
ara-unsecret Error 0
Error
kp-genetics Error 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Done
ara-improving Done 3
Error
kp-cam Error 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Error
kp-icees Error 0
Done
ara-robokop Done 0
Done
kp-chp Done 0
Done
kp-cohd Done 8
Error
kp-icees-dili Error 0
D.6_metformin-ferritin 603bda9e-d935-48c2-ae5e-616513cb84c4
Done
Done
ara-a

### Creating dataframe for workflows with PK

<br>

In [94]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_ARS_three-hop_query_results_MESH:D056487
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metformin-ferritin


In [95]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [126]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
B.0_Initial_ARS_three-hop_query_results_MESH:D056487
B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi
B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi
B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi
B.3_DILI-one-hop-from-genes
B.4_one-hop-gene-biological-process-or-activity_trapi
C.1_ChemSubstances_related_to_Disease
C.2_ChemSubstances_related_to_GeneSet
D.1_parkinsons-crohns
D.2_ssri-heart-disease
D.3_ssri-heart-disease-one-hop
D.4_tryptophan-kynurenine
D.6_metformin-ferritin


In [127]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.1_RHOBTB2': {'ara-arax': {'SEMMED Gene API',
               'infores:arax',
               'infores:rtx-kg2',
               'infores:semmeddb'},
              'ara-bte': {'SEMMED Gene API', 'infores:semmeddb'},
              'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.2_RHOBTB2_twohop': {'ara-arax': {'infores:arax',
               'infores:dgidb',
               'infores:drug-rep-hub',
               'infores:drugbank',
               'infores:molepro-kp',
               'infores:rtx-kg2',
               'infores:semmeddb'},
              'ara-bte': {'BioThings DGIdb API',
               'SEMMED Gene API',
               'infores:automat.cord19',
               'infores:automat.ctd',
               'infores:automat.hetio',
  

In [128]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [129]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'ara-improving', 'infores:improving-agent'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:lincs'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-arax', 'SEMMED Gene API'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-bte', 'SEMMED Gene API'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:improving-agent'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:spoke'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:dgidb'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:drug-rep-hub'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:semmeddb'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:molepro-kp'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:rtx-kg2'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:arax'],
 ['A.2_RHOBTB2_t

In [150]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [151]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [152]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [153]:
df2test = pd.DataFrame(df2test.unstack().T)

In [154]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,
BioThings DGIdb API,NaN,NaN,[ara-bte],"[ara-arax, ara-bte]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Clinical Risk KP API,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,[ara-bte],NaN,NaN
Multiomics Wellness KP API,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte]
MyChem.info API,NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN
SEMMED Gene API,NaN,"[ara-arax, ara-bte]",[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pmid:23779130,NaN,NaN,[ara-improving],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pmid:24356815,NaN,NaN,[ara-improving],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pmid:25184630,NaN,NaN,[ara-improving],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
#df2test.drop([''], axis=0, inplace=True)

In [136]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [155]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,
BioThings DGIdb API,NaN,NaN,[ara-bte],"[ara-arax, ara-bte]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Clinical Risk KP API,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,[ara-bte],NaN,NaN
Multiomics Wellness KP API,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte]
MyChem.info API,NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN
SEMMED Gene API,NaN,"[ara-arax, ara-bte]",[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pmid:23779130,NaN,NaN,[ara-improving],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pmid:24356815,NaN,NaN,[ara-improving],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pmid:25184630,NaN,NaN,[ara-improving],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#df2.replace([], 'None', regex=True,inplace=True)

In [20]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [156]:
df2test = df2test.rename_axis(None)

In [157]:
df2test.columns.name = None

In [158]:
df2test.fillna('', inplace=True)

In [159]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
BioThings DGIdb API,,,[ara-bte],"[ara-arax, ara-bte]",,,,,,,,,,,,,,,
Clinical Risk KP API,,,,,,,,,,,,,,[ara-bte],,,[ara-bte],,
Multiomics Wellness KP API,,,,,,,,,,,,,,,,,,,[ara-bte]
MyChem.info API,,,,[ara-arax],,,,,,,,,,,,,[ara-bte],,
SEMMED Gene API,,"[ara-arax, ara-bte]",[ara-bte],,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pmid:23779130,,,[ara-improving],,,,,,,,,,,,,,,,
pmid:24356815,,,[ara-improving],,,,,,,,,,,,,,,,
pmid:25184630,,,[ara-improving],,,,,,,,,,,,,,,,
pmid:26000489,,,[ara-improving],,,,,,,,,,,,,,,,


## Sorting The index to push infores ID's that are compliant to Column L and M in INFORES Catalog


<br>
<br>

In [160]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv")

In [161]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]
infores_catalog = infores_catalog[:331]

In [162]:
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

In [163]:
dict_map

{'BioThings DGIdb API': 'Illegal value',
 'Clinical Risk KP API': 'Illegal value',
 'Multiomics Wellness KP API': 'Illegal value',
 'MyChem.info API': 'Illegal value',
 'SEMMED Gene API': 'Illegal value',
 'Text Mining Targeted Association API': 'Illegal value',
 'icees:dili': 'Illegal value',
 'infores:aragorn-ranker-ara': 'Illegal value',
 'infores:arax': 'ARA',
 'infores:automat.cord19': 'Illegal value',
 'infores:automat.ctd': 'Illegal value',
 'infores:automat.hetio': 'Illegal value',
 'infores:automat.ontology-hierarchy': 'Illegal value',
 'infores:automat.pharos': 'Illegal value',
 'infores:automat.robokop': 'Illegal value',
 'infores:automat.uberongraph': 'Illegal value',
 'infores:bindingdb': 'External Source',
 'infores:chembl': 'External Source',
 'infores:civic': 'External Source',
 'infores:cohd': 'KP',
 'infores:connections-hypothesis': 'KP',
 'infores:cord19-scibite': 'External Source',
 'infores:ctd': 'External Source',
 'infores:dgidb': 'External Source',
 'infores:dis

In [164]:
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)

In [165]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog
BioThings DGIdb API,,,[ara-bte],"[ara-arax, ara-bte]",,,,,,,,,,,,,,,,Illegal value
Clinical Risk KP API,,,,,,,,,,,,,,[ara-bte],,,[ara-bte],,,Illegal value
Multiomics Wellness KP API,,,,,,,,,,,,,,,,,,,[ara-bte],Illegal value
MyChem.info API,,,,[ara-arax],,,,,,,,,,,,,[ara-bte],,,Illegal value
SEMMED Gene API,,"[ara-arax, ara-bte]",[ara-bte],,,,,,,,,,,,,,,,,Illegal value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pmid:23779130,,,[ara-improving],,,,,,,,,,,,,,,,,Illegal value
pmid:24356815,,,[ara-improving],,,,,,,,,,,,,,,,,Illegal value
pmid:25184630,,,[ara-improving],,,,,,,,,,,,,,,,,Illegal value
pmid:26000489,,,[ara-improving],,,,,,,,,,,,,,,,,Illegal value


In [166]:
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)

In [167]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog,sort
infores:genetics-data-provider,,,,,,,,,,,...,,,,[ara-arax],,,,,KP,KP
infores:lincs,[ara-improving],[ara-improving],"[ara-improving, ara-robokop]",[ara-improving],[ara-improving],[ara-improving],,,,,...,"[ara-improving, ara-robokop]",,,,[ara-improving],,,,KP,KP
infores:spoke,[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving],,,,,...,[ara-improving],,,[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving],KP,KP
infores:cohd,,,,,,,,,,,...,,,[kp-cohd],,,[kp-cohd],,,KP,KP
infores:connections-hypothesis,,,,[kp-chp],,,,,,,...,[kp-chp],,,,,,,,KP,KP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pmid:23779130,,,[ara-improving],,,,,,,,...,,,,,,,,,Illegal value,NaN
pmid:24356815,,,[ara-improving],,,,,,,,...,,,,,,,,,Illegal value,NaN
pmid:25184630,,,[ara-improving],,,,,,,,...,,,,,,,,,Illegal value,NaN
pmid:26000489,,,[ara-improving],,,,,,,,...,,,,,,,,,Illegal value,NaN


In [168]:
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]

In [170]:
df2test[0:20]

,Translator_Category_Complaint_to_ColL&M_InforesCatalog,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
infores:genetics-data-provider,KP,,,,,,,,,,,,,,,[ara-arax],,,,
infores:lincs,KP,[ara-improving],[ara-improving],"[ara-improving, ara-robokop]",[ara-improving],[ara-improving],[ara-improving],,,,,,"[ara-improving, ara-robokop]",,,,[ara-improving],,,
infores:spoke,KP,[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving],,,,,,[ara-improving],,,[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving]
infores:cohd,KP,,,,,,,,,,,,,,[kp-cohd],,,[kp-cohd],,
infores:connections-hypothesis,KP,,,,[kp-chp],,,,,,,,[kp-chp],,,,,,,
infores:rtx-kg2,KP,,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],,[ara-arax],[ara-arax],[ara-arax],[ara-arax],
infores:arax,ARA,,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],
infores:translator-biothings-explorer,ARA,,,[ara-bte],,[ara-bte],[ara-bte],,,,,,,,,,,[ara-bte],,[ara-bte]
infores:improving-agent,ARA,[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving],,,,,,[ara-improving],,,[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving]
infores:sider,External Source,,,,,,,,,,,,,,,,,[ara-bte],,


<br>

## Converting the Pk's to hyperlink

<br>

In [26]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [27]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [28]:
df.sort_index(inplace=True)

In [29]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 500,No Results: 200,Error: 500,Error: 500,Error: 500,Error: 598,Error: 500,Error: 598,No Results: 200,...,Error: 422,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 200,No Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,Error: 400,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,Error: 598,Results: 200
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,...,No Results: 200,Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598,No Results: 200,Error: 598,No Results: 200


<br>

### Highlight the cells In Excel

<br>

In [30]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [31]:
styled = df.style.applymap(highlight)

In [32]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 500,No Results: 200,Error: 500,Error: 500,Error: 500,Error: 598,Error: 500,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 422,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 200,No Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,Error: 598,Results: 200
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,No Results: 200,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 598,Err

In [33]:
#import xlsxwriter

In [34]:
#import xlwt

In [35]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [38]:
wks_name

'Workflow Progress Tracker_2021_09_14-00_10_17_AM'

In [ ]:
#writer = pd.ExcelWriter('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx', engine='xlsxwriter')

In [39]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


In [ ]:
### Save as excel sheet
#df.to_excel('ara_worklow_progress_tracker_Aug5.xlsx', sheet_name = 'Progress_Tracker')

In [ ]:
### save all the ids for ara submission
#with open('ara_worklow_IDs_Aug9.txt', 'w') as f:
#    print(dict_workflows, file=f)

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [40]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1d_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1e_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1f_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 500,No Results: 200,Error: 500,Error: 500,Error: 500,Error: 598,Error: 500,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 422,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 200,No Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,Error: 598,Results: 200
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,No Results: 200,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 598,Err

In [41]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,B.0_Initial_ARS_three-hop_query_results_MESH:D056487,B.1a_DILI-three-hop-from-disease-or-phenotypic-feature_trapi,B.1b_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,B.1c_DILI_three-hop-from-disease-or-phenotypic-feature_trapi,...,B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi,B.3_DILI-one-hop-from-genes,B.4_one-hop-gene-biological-process-or-activity_trapi,C.1_ChemSubstances_related_to_Disease,C.2_ChemSubstances_related_to_GeneSet,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 500,No Results: 200,Error: 500,Error: 500,Error: 500,Error: 598,Error: 500,Error: 598,No Results: 200,...,Error: 422,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,No Results: 200,No Results: 200
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,Error: 400,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,Results: 200,Error: 598,Error: 598,Error: 598,Results: 200,Error: 598,Results: 200
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 400,Results: 200,No Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-robokop,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,Error: 422,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-unsecret,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 504,Error: 504,Error: 504,No Results: 200,Results: 200
kp-cam,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,...,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502,Error: 502
kp-chp,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,...,No Results: 200,Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598,No Results: 200,Error: 598,No Results: 200


<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [42]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [43]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [44]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker_2021_09_14-00_10_17_AM' id:1670904860>

In [45]:
spreadsheet_key

'1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'

In [46]:
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [47]:
pwd

'/Users/priyash/Documents/GitHub/minihackathons/Notebooks'

In [48]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
sh = wksh.worksheet(wks_name)

In [49]:
sh

<Worksheet 'Workflow Progress Tracker_2021_09_14-00_10_17_AM' id:1670904860>

In [50]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [51]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1670904860,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [52]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1670904860,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1670904860,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'r

In [53]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1670904860,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1670904860,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bo

In [54]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('B2:{}18', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1670904860,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1670904860,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 1,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
    

In [55]:
sh

<Worksheet 'Workflow Progress Tracker_2021_09_14-00_10_17_AM' id:1670904860>

In [56]:
# Select a range
cell_list = sh.range('B17:Y19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_14-00_10_17_AM'!B17:Y19",
 'updatedRows': 3,
 'updatedColumns': 24,
 'updatedCells': 72}

### Push Dataframe 2

<br>

In [171]:
wks2 = 'edge_attribute_source_' + date

In [172]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_09_14-06_05_13_AM' id:1968643229>